In [ ]:
import csv
import os
import pandas as pd
import re
import time
from datetime import datetime
from pathlib import *
import random
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from pyvirtualdisplay import Display
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

# Start a virtual display
display = Display(visible=0, size=(1920, 1080))
display.start()


import pytz
# Define the Vietnam timezone
local_tz = pytz.timezone('Asia/Ho_Chi_Minh')
# Get the current time in UTC
now_utc = datetime.now(pytz.utc)
# Convert UTC time to local time
now = now_utc.astimezone(local_tz).date().strftime('%Y-%m-%d')

# Chrome
def chrome_drive():

    # Configure Chrome options
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')  # Run in headless mode
    options.add_argument('--disable-gpu')  # Disable GPU hardware acceleration
    options.add_argument('--no-sandbox')  # Bypass OS security model
    options.add_argument('--disable-dev-shm-usage')  # Overcome limited resource problems

    # Create a driver instance
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    # Disable the "Chrome is being controlled by automated test software" notification
    options.add_experimental_option("excludeSwitches", ["enable-automation"])

    # # Disable the "navigator.webdriver" property
    options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

    # Disable the "Chrome is being controlled by automated test software" banner
    options.add_argument("--disable-blink-features=AutomationControlled")

    # Maximize the window to avoid fingerprinting based on screen resolution
    options.add_argument("start-maximized")

    # Disabling the Automation Extension can help prevent detection as an automated script and increase the chances of
    # successfully completing your automation tasks.
    options.add_experimental_option('useAutomationExtension', False)

    # This argument tells the browser to ignore any SSL certificate errors that may occur while accessing a website,
    # which is useful when testing on a site with a self-signed or invalid SSL certificate. Without this argument,
    # the browser will display a warning message about the certificate and require manual confirmation to proceed.
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--ignore-ssl-errors=yes')

    # wait for the page to be fully loaded before proceeding
    # options.page_load_strategy = 'normal'  # 'none', 'eager', or 'normal'

    prefs={
        "disable-transitions": True,
        "profile.managed_default_content_settings.images": 2,
        "profile.default_content_setting_values.notifications": 2
    }
    options.add_experimental_option("prefs", prefs)

    return driver


class TZ:
    def tz(self, link_check: list, restart_link: str):

        driver = chrome_drive()

        wait = WebDriverWait(driver, 20)

        # Define the base path
        base_path = '/content'
        output_dir = os.path.join(base_path, f"{now}")
        output_img = os.path.join(output_dir, 'img_tz')

        def record():
            output_dir = os.path.join(base_path, f"{now}")

            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            file_path = os.path.join(output_dir, f"8-tz-{now}.csv")

            with open(file_path, "a") as file:
                writer = csv.DictWriter(file,
                                        fieldnames=["Product_Name", "Ton_Kho", "Gia_Niem_Yet", "Gia_Khuyen_Mai","+VNPAY", "Date",
                                                    "Khuyen_Mai", "vnpay", "Link"], delimiter=";")
                if os.stat(file_path).st_size == 0:
                    writer.writeheader()
                for row in data_list:
                    writer.writerow(row)
        
        def screen_shot(product_name):
          if not os.path.exists(output_img):
              os.makedirs(output_img)
          product_name_new=product_name.replace("/", "")
          driver.fullscreen_window()
          driver.set_window_size(1920, 2080)
          # Scroll up by 100 pixels
          driver.execute_script("window.scrollBy(0, -100);")
          driver.get_screenshot_as_png()
          driver.save_screenshot(f"{output_img}/{product_name_new}_{now_utc.astimezone(local_tz)}.png")

        def check_cash_discount(gia_khuyen_mai_raw):

            # 2 Check xem có khuyến mại bằng tiền mặt giảm trực tiếp không
            try:
                # This is when a direct discount is applied
                driver.find_element(By.CLASS_NAME, "choose")
            except NoSuchElementException:
                gia_khuyen_mai_new = gia_khuyen_mai_raw
                return gia_khuyen_mai_new
            else:
                option_km_them = driver.find_elements(By.CLASS_NAME, "choose")
                for i in option_km_them:
                    # if ("đ" in i.text.lower()) and ("ava" not in i.text.lower()) and ("xanh" not in i.text.lower()):
                    if ("đ" in i.text.lower()) and ("ava" not in i.text.lower()) and ("xanh" not in i.text.lower()) and ("giảm" in i.text.lower()):
                        # i.click()
                        # time.sleep(1)
                        try:
                            km_them = int(
                                driver.find_element(By.CLASS_NAME, "choose").text.replace("Giảm giá ", "")
                                .replace("đ", "").replace("*", "").replace(".", "").replace(",", "").replace('₫', ''))
                            print(f"km_them: {km_them}")
                        except ValueError:
                            # This happends only when there is no cash discount, but the channel
                            # offers promotion in kind instead
                            km_them = 0
                            print(f"km_them: {km_them}")
                        gia_khuyen_mai_new = int(gia_khuyen_mai_raw.replace("đ", "").replace(".", "").replace("₫", "")) \
                        - int(km_them)

                        print(f"mai: {gia_khuyen_mai_new}")
                        return gia_khuyen_mai_new
                    else:
                        gia_khuyen_mai_new = gia_khuyen_mai_raw
                        return gia_khuyen_mai_new
            # finally:
            #     return gia_khuyen_mai_new

        def check_shock_price():
            # Check for special case where price is so low that no other promotion is applied
            try:
                # gia_soc = driver.find_element(By.CSS_SELECTOR, ".bs_title strong").text.replace(" *", "")
                # From Feb 18th, 2023
                gia_soc = driver.find_element(By.CSS_SELECTOR, ".bs_price strong").text.replace(" *", "")
            except NoSuchElementException:
                try:
                    # This class started on August 19th, 2022
                    gia_soc = driver.find_element(By.CSS_SELECTOR, ".oo-left strong").text.replace(" *", "")
                except NoSuchElementException:
                    return None
                else:
                    gia_khuyen_mai = gia_soc + "soc"
                    gia_niem_yet = driver.find_element(By.CSS_SELECTOR, ".oo-left em").text.replace(" *", "")
                    # print(gia_khuyen_mai)
                    # print(gia_niem_yet)
                    return gia_khuyen_mai, gia_niem_yet
            else:
                # return gia_soc + "soc"
                gia_khuyen_mai = gia_soc + "soc"
                gia_niem_yet = driver.find_element(By.CSS_SELECTOR, ".bs_price em").text.replace(" *", "")
                # print(gia_khuyen_mai)
                # print(gia_niem_yet)
                return gia_khuyen_mai, gia_niem_yet

        def check_price():
            # wait.until(EC.visibility_of_element_located((By.TAG_NAME, "h1")))
            try:
                driver.find_element(By.TAG_NAME, "h1")

            except NoSuchElementException:
                product_name = f"Please double check the link: {link}"
                print(product_name)
                ton_kho = "No"
                gia_niem_yet = 0
                gia_khuyen_mai = 0
                khuyen_mai = ""
                vnpay = "0"
                gia_khuyen_mai_vnpay=0
                new_data={"Product_Name": product_name, "Ton_Kho": ton_kho, "Gia_Niem_Yet": gia_niem_yet,
                          "Gia_Khuyen_Mai": gia_khuyen_mai, "+VNPAY": gia_khuyen_mai_vnpay, "Date": now,
                          "Khuyen_Mai": khuyen_mai, "vnpay": vnpay,
                          "Link": link}
                data_list.append(new_data)

            else:
                # # From Dec 1st, 2023, click to color button to know what is the best price
                # color_options = driver.find_elements(By.XPATH, "(//div[@class='color-sp']/ul/li)")
                # num_color_options = len(color_options)
                # for num in reversed(range(num_color_options)):
                #     driver.execute_script("arguments[0].click();", color_options[num])
                #     time.sleep(2)
                #     color_options = driver.find_elements(By.XPATH, "(//div[@class='color-sp']/ul/li)")

                # If the product has the class="capacity", use this capacity to add to product name
                try:
                    driver.find_element(By.CLASS_NAME, "capacity")

                except NoSuchElementException:
                    product_name = link[link.rindex("/") + 1:].replace("-", " ").replace("iphone", "iPhone").replace("pro",
                                                                                                                     "Pro").replace(
                        "gb", "GB").replace("tb", "TB").replace("max", "Max").replace("macbook", "MacBook").replace("ipad",
                                                                                                                    "iPad").replace(
                        "apple", "Apple").replace("watch", "Watch")
                else:
                    capacities = driver.find_elements(By.CSS_SELECTOR, ".capacity ul > li")
                    capacity_text = str
                    for capacity in capacities:
                        if capacity.get_attribute("class") == "active":
                            capacity_text = capacity.text
                        else:
                            pass
                    try:
                        product_name = driver.find_element(By.TAG_NAME, "h1").text.replace(" Mới", "") + " " + capacity_text
                    except TypeError:
                        product_name = link[link.rindex("/") + 1:]
                print(product_name)

                try:
                    driver.find_element(By.CLASS_NAME, "btn-buy")
                except NoSuchElementException:
                    ton_kho = "No"
                else:
                    ton_kho = "Yes"
                print(ton_kho)

                try:
                    driver.find_element(By.CSS_SELECTOR, ".price")
                except NoSuchElementException:
                    try:
                        driver.find_element(By.CSS_SELECTOR, ".bs_price")
                    except NoSuchElementException:
                        ton_kho = "No"
                        gia_niem_yet = "not trading"
                        gia_khuyen_mai = "not trading"
                        khuyen_mai = "not trading"
                        vnpay = "0"
                        gia_khuyen_mai_vnpay = 0
                        new_data = {"Product_Name": product_name, "Ton_Kho": ton_kho, "Gia_Niem_Yet": gia_niem_yet,
                                    "Gia_Khuyen_Mai": gia_khuyen_mai, "+VNPAY": gia_khuyen_mai_vnpay, "Date": now, "Khuyen_Mai": khuyen_mai, "vnpay": vnpay,
                                    "Link": link}
                        data_list.append(new_data)
                    else:
                        # Apply from June 27th, 2023
                        shock = check_shock_price()
                        if shock:
                            gia_khuyen_mai, gia_niem_yet = shock
                            vnpay = 0
                else:
                    gia_raw = driver.find_element(By.CSS_SELECTOR, ".price").text
                    # print(gia_raw)
                    if gia_raw.count("₫") == 2:
                        gia_niem_yet = gia_raw[
                                       (gia_raw.index("₫") + 2):(gia_raw.index("₫", gia_raw.index("₫") + 2) + 1)].replace(
                            "*", "")
                        gia_khuyen_mai_raw = gia_raw[:(gia_raw.index("₫") + 1)].replace("*", "")
                        gia_khuyen_mai = check_cash_discount(gia_khuyen_mai_raw)
                    else:
                        try:
                            gia_khuyen_mai_raw = driver.find_element(By.CLASS_NAME, "giamsoc-ol-price").text.replace("*", "")
                            gia_khuyen_mai = check_cash_discount(gia_khuyen_mai_raw)
                            gia_niem_yet = driver.find_element(By.CSS_SELECTOR, ".price").text.replace("*", "")
                        except NoSuchElementException:
                            gia_niem_yet = driver.find_element(By.CSS_SELECTOR, ".price").text.replace("*", "")
                            gia_khuyen_mai_raw = gia_niem_yet
                            gia_khuyen_mai = check_cash_discount(gia_khuyen_mai_raw)
                    print(f"gia_niem_yet : {gia_niem_yet}")
                    print(f"gia_khuyen_mai: {gia_khuyen_mai}")

                    # From early June 2022, Topzone has dedicated line for VNPAYQR already, these line to record VNPAYQR
                    try:
                        driver.find_element(By.ID, "vnpayqr")
                    except NoSuchElementException:
                        vnpay = "0"
                    else:
                        payment_raws = driver.find_elements(By.CLASS_NAME, "campaign-option")
                        for payment in payment_raws:
                            if "vnpayqr" in payment.get_attribute("data-campaignname").lower():
                                # vnpay = payment.find_element(By.TAG_NAME, "b b").text.replace(".", "").replace("₫", "")
                                vnpay = payment.find_element(By.XPATH, "//figure[@class='vnpayqr']/following-sibling::b/b").text.replace(".", "").replace("₫", "")
                                print(f"vnpay {vnpay}")
                            else:
                                pass

                # Promotion in text
                try:
                    driver.find_element(By.ID, "promotion-detail")
                except NoSuchElementException:
                    khuyen_mai = ""
                else:
                    khuyen_mai = driver.find_element(By.ID, "promotion-detail").text

                # need it for every morning price with a Chau
                def vnpay_tz(km_TZ):
                    # Check if VNPAYQR is already recorded in Topzone dataframe
                    if int(vnpay) > 0:
                        try:
                            # Calculate the total discount amount
                            total_discount = int(km_TZ) - vnpay
                        except ValueError:
                            total_discount = km_TZ
                    else:
                        # If VNPAY is not recorded, check if it's in the promotion note
                        promotion_note = khuyen_mai.lower()

                        try:
                            gia_khuyen_mai = km_TZ.replace(".", "").replace(",", "").replace("đ", "").replace("₫","")
                        except ValueError:
                            gia_khuyen_mai = 0
                        except AttributeError:
                            gia_khuyen_mai = km_TZ

                        if type(promotion_note) != float:
                            if "vnpay" in promotion_note:
                                promotions = promotion_note.split("\n")
                                for promotion in promotions:
                                    if "vnpay" in promotion and "%" in promotion:
                                        percent_discount = int(
                                            promotion[(promotion.index("%") - 2):promotion.index("%")])
                                        max_raw_discount = promotion[(promotion.index("đa") + 3):]
                                        max_discount = int(
                                            max_raw_discount[:max_raw_discount.index("đ")].replace(".", "").replace(
                                                ",", ""))
                                    elif "vnpay" in promotion and "%" not in promotion:
                                        if "đơn hàng từ" not in promotion:
                                            try:
                                                max_raw_discount = promotion[(promotion.index("đến") + 3):]

                                            except ValueError:
                                                max_raw_discount = promotion[(promotion.index("đa") + 3):]
                                            max_discount = int(
                                                max_raw_discount[:max_raw_discount.index("đ")].replace(".",
                                                                                                       "").replace(
                                                    ",",
                                                    "").replace('₫', ''))
                                        elif "đơn hàng từ" in promotion:
                                            condition = int(promotion[(promotion.index("đơn hàng từ") + 12):(
                                                promotion.index("triệu"))]) * 1000000
                                            if int(gia_khuyen_mai.replace("soc", "")) >= condition:
                                                try:
                                                    max_raw_discount = promotion[(promotion.index("đa") + 3):]
                                                except ValueError:
                                                    max_raw_discount = promotion[(promotion.index("ngay") + 5):]
                                                max_discount = int(
                                                    max_raw_discount[:max_raw_discount.index("k")]) * 1000
                                            else:
                                                max_discount = 0
                                        percent_discount = 100

                                # Calculate the total discount amount
                                total_discount_1 = int(str(km_TZ).replace("₫", "").replace(".","").replace("soc","")) * ((100 - percent_discount) / 100)
                                total_discount_2 = int(str(km_TZ).replace("₫", "").replace(".","").replace("soc","")) - max_discount
                                total_discount = max(total_discount_1, total_discount_2)
                            else:
                                total_discount = km_TZ
                        else:
                            total_discount = km_TZ
                    return total_discount

                gia_khuyen_mai_vnpay = vnpay_tz(gia_khuyen_mai)

                new_data = {"Product_Name": product_name, "Ton_Kho": ton_kho,
                            "Gia_Niem_Yet": gia_niem_yet.replace("Giá dự kiến:", ""),
                            "Gia_Khuyen_Mai": gia_khuyen_mai, "+VNPAY": gia_khuyen_mai_vnpay, "Date": now,
                            "Khuyen_Mai": khuyen_mai, "vnpay": vnpay, "Link": link}
                data_list.append(new_data)

                screen_shot(product_name)


        self.link_check = link_check
        self.restart_link = restart_link

        for link in self.link_check[self.link_check.index(self.restart_link):]:
            print(link)
            data_list=[]
            try:
                driver.get(link)
                time.sleep(5)
                check_price()
                record()
            except (TimeoutException, WebDriverException):
                print(datetime.now())
                print(f"Start again from link: {link}")
                self.restart_link = link
                self.tz(self.restart_link)

        driver.quit()